TVAE Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `TVAE` model, including how to:

-   Create an instance of `TVAE`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `TVAE` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is TVAE?
--------------

The `sdv.tabular.TVAE` model is based on the VAE-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `TVAE` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `tvae` installed on your
system. If you have not done it yet, please install `tvae` now by
executing the command `pip install sdv` in a terminal.

</div>

In [1]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they where offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

T   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `TVAE` to learn this data and then sample synthetic data
about new students to see how well de model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.TVAE` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [2]:
from sdv.tabular import TVAE

model = TVAE()
model.fit(data)

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying TVAESynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [3]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [4]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17322,M,55.684265,69.997331,Science,62.564916,Others,False,0,83.334043,Mkt&Fin,64.665410,NaN,True,2020-03-27,2020-11-08,12.0
1,17414,M,58.625805,78.864361,Arts,62.414653,Others,False,0,61.495522,Mkt&Fin,59.932214,26446.055454,False,NaT,NaT,12.0
2,17410,M,64.852171,78.435276,Science,65.605048,Comm&Mgmt,False,0,76.044449,Mkt&HR,62.023110,NaN,True,NaT,NaT,12.0
3,17298,M,69.769258,77.096665,Science,57.939864,Others,False,0,59.638787,Mkt&HR,64.114351,25995.482399,True,2020-03-12,2020-08-17,12.0
4,17428,M,64.337031,76.564920,Arts,64.608248,Comm&Mgmt,False,0,83.517522,Mkt&HR,64.618888,NaN,False,NaT,NaT,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [5]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `TVAE.load` method, and then you are ready to sample new data
from the loaded instance:

In [6]:
loaded = TVAE.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `tvae` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [7]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [8]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
9,17395,M,65.129457,70.037512,Science,58.552571,Sci&Tech,False,0,74.824963,Mkt&Fin,66.753046,49758.158935,False,NaT,NaT,12.0
39,17395,M,56.570191,80.077740,Science,60.907843,Comm&Mgmt,False,0,77.008217,Mkt&HR,59.152451,49354.361922,True,NaT,NaT,12.0
51,17395,M,78.788869,65.946024,Commerce,60.680985,Others,False,0,60.655187,Mkt&Fin,61.617319,NaN,True,2020-03-16,NaT,12.0
106,17395,M,56.908531,67.137670,Science,60.528240,Sci&Tech,False,0,80.246539,Mkt&HR,64.214132,NaN,False,NaT,NaT,12.0
150,17395,M,60.160797,66.175834,Science,61.655285,Comm&Mgmt,False,0,62.673138,Mkt&HR,58.544073,23011.248811,False,NaT,NaT,12.0
198,17395,M,58.818272,58.331242,Commerce,62.546009,Comm&Mgmt,False,0,64.117797,Mkt&Fin,67.966149,NaN,False,2020-07-28,NaT,6.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [9]:
model = TVAE(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,77.791417,60.713517,Commerce,63.246426,Sci&Tech,False,0,74.837984,Mkt&HR,64.943827,NaN,True,NaT,NaT,6.0
1,1,F,80.757047,66.203647,Arts,60.274467,Sci&Tech,False,0,57.095676,Mkt&Fin,70.833625,NaN,True,NaT,NaT,6.0
2,2,M,80.830291,64.583290,Arts,61.878889,Sci&Tech,False,0,63.229952,Mkt&HR,68.349610,NaN,True,NaT,NaT,6.0
3,3,F,59.421203,66.855202,Commerce,65.693179,Others,False,0,66.403845,Mkt&HR,68.615932,NaN,False,2020-06-18,NaT,6.0
4,4,F,79.689830,65.662571,Arts,63.924379,Sci&Tech,False,0,68.937502,Mkt&HR,68.310832,27815.032856,False,NaT,NaT,6.0


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [10]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [11]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [12]:
model = TVAE(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"67904 Bowen Ways Suite 290\nHernandezville, VT...",M,56.300018,64.956066,Science,70.213852,Sci&Tech,True,0,82.839569,Mkt&HR,55.628393,34655.149820,True,NaT,2020-04-13,6.0
1,1,"731 Valenzuela Knoll\nNew Maria, SD 66256",M,62.709856,67.505749,Science,73.960294,Sci&Tech,False,1,64.145517,Mkt&HR,68.363943,NaN,False,NaT,2020-04-21,12.0
2,2,"427 Smith Knolls Apt. 645\nLake Kathrynville, ...",M,58.915830,64.747642,Arts,67.196981,Sci&Tech,True,1,65.331207,Mkt&HR,62.874127,29162.568885,False,NaT,2020-03-27,3.0
3,3,"793 Christopher Branch\nRonaldville, MD 38677",M,72.791580,69.353760,Science,72.403968,Comm&Mgmt,False,0,67.092526,Mkt&HR,66.329499,NaN,True,NaT,2020-04-19,6.0
4,4,"731 Valenzuela Knoll\nNew Maria, SD 66256",M,60.266297,61.762514,Science,75.688405,Sci&Tech,True,0,84.881349,Mkt&HR,61.079857,29162.293327,True,NaT,2020-04-06,6.0


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [13]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [14]:
model = TVAE(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [15]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"57730 Christopher Crest\nWest Andrewberg, OK 6...",M,63.784789,67.984261,Arts,67.079820,Comm&Mgmt,True,0,88.192394,Mkt&Fin,58.480276,33823.345961,False,NaT,NaT,12.0
1,1,"5801 Rose Summit Suite 653\nWest Amber, NV 98694",M,65.398313,65.102538,Science,70.925973,Others,True,0,84.482310,Mkt&Fin,65.088168,33597.741129,False,NaT,NaT,3.0
2,2,"751 Davis Prairie Suite 999\nLake Samuelfurt, ...",M,78.886780,65.160893,Science,69.398196,Comm&Mgmt,True,1,78.760367,Mkt&Fin,66.592766,27712.998291,False,2020-07-07,NaT,3.0
3,3,"6698 Price Squares Suite 190\nSouth Stephanie,...",M,62.823696,69.480274,Science,67.707133,Sci&Tech,True,0,85.479024,Mkt&Fin,59.429059,26584.956291,False,2020-01-19,2020-08-09,3.0
4,4,"1426 Rowe Path\nLake Ericshire, AL 59620",M,74.589613,65.439834,Arts,59.145945,Comm&Mgmt,True,1,95.567033,Mkt&Fin,56.019989,30203.488133,False,NaT,NaT,3.0


Which means that none of the original addresses can be found in the
sampled data:

In [16]:
data_pii.address.isin(new_data_pii.address).sum()

0

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.